In [3]:
import json

# Paths to the input and output files
data_file = 'attrebute_val.data'
solution_file = 'attrebute_val.solution'
output_file = 'val_fine_tuning_dataset.jsonl'

# Load JSONL data
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

# Create prompt-completion pairs for fine-tuning
def create_fine_tuning_dataset(data, solution):
    fine_tuning_pairs = []
    
    # Create a dictionary from solution file using 'indoml_id' as key
    solution_dict = {item['indoml_id']: item for item in solution}
    
    for item in data:
        indoml_id = item['indoml_id']
        if indoml_id in solution_dict:
            # Shorter, concise prompt
            prompt = (
                "Given a product title and optional details (store and manufacturer), predict the brand and "
                "5 hierarchical categories (L0 to L4). The values may or may not be present in the title. "
                "Be prepared for unseen brands and categories.\n\n"
                f"Title: {item['title']}\n"
                f"Store: {item['store']}\n"
                f"Manufacturer: {item.get('details_Manufacturer', '')}"
            )
            
            # Get the completion from the corresponding solution record
            solution_item = solution_dict[indoml_id]
            completion = (
                f"Brand: {solution_item['details_Brand']}, L0_category: {solution_item['L0_category']}, "
                f"L1_category: {solution_item['L1_category']}, L2_category: {solution_item['L2_category']}, "
                f"L3_category: {solution_item['L3_category']}, L4_category: {solution_item['L4_category']}"
            )
            
            # Create the prompt-completion pair
            fine_tuning_pairs.append({"prompt": prompt, "completion": completion})
    
    return fine_tuning_pairs

# Save fine-tuning dataset to JSONL format
def save_jsonl(data, file_path):
    with open(file_path, 'w') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')

# Load the data and solution files
attribute_train_data = load_jsonl(data_file)
attribute_train_solution = load_jsonl(solution_file)

# Create fine-tuning dataset
fine_tuning_dataset = create_fine_tuning_dataset(attribute_train_data, attribute_train_solution)

# Save to output file
save_jsonl(fine_tuning_dataset, output_file)

print(f"Fine-tuning dataset saved to {output_file}")


Fine-tuning dataset saved to val_fine_tuning_dataset.jsonl


In [1]:
import json
import random

# File paths
input_file = "val_fine_tuning_dataset.jsonl"  # Input JSONL file
output_file = 'sample_val_fine_tuning_dataset.jsonl'  # Output sample JSONL file

# Parameters
sample_size = 20  # Number of samples to extract
random_sampling = True  # Set to False to take the first 'n' samples

# Load JSONL data
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

# Save JSONL data
def save_jsonl(data, file_path):
    with open(file_path, 'w') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')

# Sample data
def sample_jsonl_data(data, sample_size, random_sampling):
    if random_sampling:
        return random.sample(data, min(sample_size, len(data)))
    else:
        return data[:sample_size]

# Load the large JSONL file
data = load_jsonl(input_file)

# Take a sample of the data
sampled_data = sample_jsonl_data(data, sample_size, random_sampling)

# Save the sampled data to a new JSONL file
save_jsonl(sampled_data, output_file)

print(f"Sampled {len(sampled_data)} records saved to {output_file}")


Sampled 20 records saved to sample_val_fine_tuning_dataset.jsonl
